<a href="https://colab.research.google.com/github/jcyeh/dl_scripts/blob/master/fizzbuzz_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FizzBuzz in TensorFlow (v2.0.0)

This Jupyter notebook demonstrates the FizzBuzz code running on TPU device

In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

The TPU initialization code has to be at the beginning of your program.

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
# Represent each input by an array of its binary digits.
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

In [ ]:
# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]
def fizz_buzz_encode(i):
    if i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5 == 0: return np.array([0, 0, 1, 0])
    elif i % 3 == 0: return np.array([0, 1, 0, 0])
    else: return np.array([1, 0, 0, 0])

In [ ]:
NUM_DIGITS = 10

x_train = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
y_train = np.array([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

x_test = np.array([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
y_test = np.array([fizz_buzz_encode(i) for i in range(1, 101)])

In [ ]:
# def get_dataset(batch_size=200):
#   datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True,
#                              try_gcs=True)
#   mnist_train, mnist_test = datasets['train'], datasets['test']

#   def scale(image, label):
#     image = tf.cast(image, tf.float32)
#     image /= 255.0

#     return image, label

#   train_dataset = mnist_train.map(scale).shuffle(10000).batch(batch_size)
#   test_dataset = mnist_test.map(scale).batch(batch_size)

#   return train_dataset, test_dataset

In [ ]:
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Dense(1000, input_shape=(10,), activation='relu'),
       tf.keras.layers.Dense(4, activation='softmax')])

In [ ]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

# train_dataset, test_dataset = get_dataset()

# model.fit(train_dataset,
#           epochs=5,
#           validation_data=test_dataset)

In [ ]:
start = time.time()
#model.fit(x_train, y_train, batch_size=64, nb_epoch=120, verbose=0)
model.fit(x_train, batch_size=64, epochs=120, validation_data=y_train)
end = time.time()
print("Traing Time: {0}s".format(end-start))

In [ ]:
start = time.time()
result = model.evaluate(x_test, y_test, verbose=0)
end = time.time()
print("Acc: {0:.2f} ".format(result[1]))
print("Evaluation Time: {0}s".format(end-start))